In [3]:
import pandas as pd
from datetime import datetime, timedelta

# 엑셀 파일 읽기
df_crrt = pd.read_excel('CRRT_setting.xlsx')

# CSV 파일 읽기 (첫 50개 행만)
df_merged = pd.read_csv('merged_table_valid.csv', nrows=50)

# 날짜/시간 컬럼을 datetime 형식으로 변환
df_crrt['[간호기록]기록작성일시'] = pd.to_datetime(df_crrt['[간호기록]기록작성일시'])
df_merged['Time'] = pd.to_datetime(df_merged['Time'], format='%Y-%m-%d %H:%M')

# 결과를 저장할 리스트
results = []

# 시간 차이 임계값 설정 (예: 10분)
time_threshold = timedelta(minutes=10)

# 필요한 컬럼들
required_columns = ['BFR', 'Pre', 'Replace', 'Dialysate']

# merged_table_valid.csv의 각 행에 대해
for _, merged_row in df_merged.iterrows():
    # 필요한 모든 컬럼에 데이터가 있는지 확인
    if all(pd.notna(merged_row[col]) for col in required_columns):
        merged_time = merged_row['Time']
        
        # CRRT_setting.xlsx에서 가장 가까운 시간 찾기
        closest_crrt_row = min(df_crrt.itertuples(), key=lambda x: abs(x._5 - merged_time))
        
        # 시간 차이가 임계값 이내인 경우에만 결과에 추가
        if abs(closest_crrt_row._5 - merged_time) <= time_threshold:
            results.append({
                'Merged_Time': merged_time,
                'CRRT_Time': closest_crrt_row._5,
                'Time_Difference': abs(closest_crrt_row._5 - merged_time),
                'Merged_Data': merged_row.to_dict(),
                'CRRT_Data': closest_crrt_row._asdict()
            })

# 결과 출력
for result in results:
    print(f"Merged Time: {result['Merged_Time']}")
    print(f"CRRT Time: {result['CRRT_Time']}")
    print(f"Time Difference: {result['Time_Difference']}")
    print("Merged Data:", {k: v for k, v in result['Merged_Data'].items() if k in ['Time', 'PT_ID', 'CRRT_type', 'BFR', 'Pre', 'Replace', 'Dialysate', 'UF']})
    print("CRRT Data:", {k: v for k, v in result['CRRT_Data'].items() if k in ['_5', 'Entity', 'Attribute', 'Value']})
    print("-" * 50)

# 결과를 CSV 파일로 저장
results_df = pd.DataFrame(results)
results_df.to_csv('matched_data.csv', index=False)
print("Results saved to 'matched_data.csv'")

ValueError: unconverted data remains when parsing with format "%Y-%m-%d %H:%M": ":00", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [15]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# 데이터 로드
crrt_setting = pd.read_excel('CRRT_setting.xlsx')
merged_table = pd.read_csv('merged_table_valid.csv')

# 날짜/시간 형식 변환
crrt_setting['[간호기록]기록작성일시'] = pd.to_datetime(crrt_setting['[간호기록]기록작성일시'])
merged_table['Time'] = pd.to_datetime(merged_table['Time'])

# 필요한 컬럼만 선택
crrt_setting = crrt_setting[['[간호기록]기록작성일시', 'Attribute', 'Value']]
merged_table = merged_table[['Time', 'PT_ID', 'CRRT_type', 'BFR', 'Pre', 'Replace', 'Dialysate']]

# 단위 제거 및 숫자로 변환
for col in ['BFR', 'Pre', 'Replace', 'Dialysate']:
    merged_table[col] = merged_table[col].str.extract('(\d+)').astype(float)

# 매칭 함수
def find_match(row, crrt_data):
    time = row['Time']
    bfr = row['BFR']
    pre = row['Pre']
    replace = row['Replace']
    dialysate = row['Dialysate']
    
    # 시간 범위 설정 (예: 3시간)
    time_range = timedelta(hours=20)
    
    # 시간 범위 내의 CRRT 설정 데이터 필터링
    filtered_crrt = crrt_data[(crrt_data['[간호기록]기록작성일시'] >= time - time_range) &
                              (crrt_data['[간호기록]기록작성일시'] <= time + time_range)]
    
    # 각 항목별 매칭 확인
    bfr_match = filtered_crrt[(filtered_crrt['Attribute'] == '투석 blood flow rate') & (filtered_crrt['Value'] == bfr)]
    pre_match = filtered_crrt[(filtered_crrt['Attribute'] == '투석 replacement(전희석)') & (filtered_crrt['Value'] == pre)]
    replace_match = filtered_crrt[(filtered_crrt['Attribute'] == '투석 replacement(후희석)') & (filtered_crrt['Value'] == replace)]
    dialysate_match = filtered_crrt[(filtered_crrt['Attribute'] == '투석 dialysate') & (filtered_crrt['Value'] == dialysate)]
    
    # 매칭 개수 확인
    match_count = sum([len(bfr_match) > 0, len(pre_match) > 0, len(replace_match) > 0, len(dialysate_match) > 0])
    
    if match_count >= 2:
        return True
    return False

# 매칭 적용
merged_table['Matched'] = merged_table.apply(lambda row: find_match(row, crrt_setting) if pd.notnull(row['BFR']) and pd.notnull(row['Pre']) and pd.notnull(row['Replace']) and pd.notnull(row['Dialysate']) else False, axis=1)

# 결과 출력
matched_data = merged_table[merged_table['Matched']]
print(f"총 {len(matched_data)}개의 매칭된 데이터를 찾았습니다.")
print(matched_data)

# 결과 저장
matched_data.to_csv('matched_crrt_data.csv', index=False)

<>:19: SyntaxWarning: invalid escape sequence '\d'
<>:19: SyntaxWarning: invalid escape sequence '\d'
C:\Users\SNUH\AppData\Local\Temp\ipykernel_18936\2881163380.py:19: SyntaxWarning: invalid escape sequence '\d'
  merged_table[col] = merged_table[col].str.extract('(\d+)').astype(float)


총 0개의 매칭된 데이터를 찾았습니다.
Empty DataFrame
Columns: [Time, PT_ID, CRRT_type, BFR, Pre, Replace, Dialysate, Matched]
Index: []


In [17]:
import pandas as pd

# 데이터 로드
crrt_setting = pd.read_excel('CRRT_setting.xlsx')
merged_table = pd.read_csv('merged_table_valid.csv')

# 날짜/시간 형식 변환
crrt_setting['[간호기록]기록작성일시'] = pd.to_datetime(crrt_setting['[간호기록]기록작성일시'])
merged_table['Time'] = pd.to_datetime(merged_table['Time'])

# CRRT_setting 데이터의 시간을 정렬하고 중복 제거
crrt_times = crrt_setting['[간호기록]기록작성일시'].sort_values().drop_duplicates()

# merged_table 데이터의 시간을 정렬하고 중복 제거
merged_times = merged_table['Time'].sort_values().drop_duplicates()

print("CRRT_setting.xlsx의 '[간호기록]기록작성일시' 데이터 (처음 20개):")
print(crrt_times.head(20).to_string(index=False))
print("\n총 고유 시간 개수:", len(crrt_times))
print("\n시간 범위:")
print(f"시작: {crrt_times.min()}")
print(f"종료: {crrt_times.max()}")

print("\n\nmerged_table_valid.csv의 'Time' 데이터 (처음 20개):")
print(merged_times.head(20).to_string(index=False))
print("\n총 고유 시간 개수:", len(merged_times))
print("\n시간 범위:")
print(f"시작: {merged_times.min()}")
print(f"종료: {merged_times.max()}")

# 두 데이터셋의 시간 범위 비교
print("\n\n두 데이터셋의 시간 범위 비교:")
print(f"CRRT_setting: {crrt_times.min()} to {crrt_times.max()}")
print(f"merged_table: {merged_times.min()} to {merged_times.max()}")

# 시간 범위 겹치는지 확인
overlap = (crrt_times.min() <= merged_times.max()) and (merged_times.min() <= crrt_times.max())
print(f"\n시간 범위 겹침: {'예' if overlap else '아니오'}")

# 결과를 CSV 파일로 저장
crrt_times.to_csv('crrt_setting_times.csv', index=False, header=['Time'])
merged_times.to_csv('merged_table_times.csv', index=False, header=['Time'])
print("\n전체 시간 데이터를 'crrt_setting_times.csv'와 'merged_table_times.csv' 파일로 저장했습니다.")

CRRT_setting.xlsx의 '[간호기록]기록작성일시' 데이터 (처음 20개):
2021-07-06 07:00:00
2021-07-06 08:25:00
2021-07-06 11:56:00
2021-07-06 16:00:00
2021-07-07 00:00:00
2021-07-07 08:00:00
2021-07-07 14:00:00
2021-07-08 21:15:00
2021-07-08 21:30:00
2021-07-08 23:00:00
2021-07-09 00:00:00
2021-07-09 01:08:00
2021-07-09 01:30:00
2021-07-09 05:23:00
2021-07-09 06:00:00
2021-07-09 06:30:00
2021-07-09 07:00:00
2021-07-09 07:30:00
2021-07-09 08:00:00
2021-07-09 10:00:00

총 고유 시간 개수: 17296

시간 범위:
시작: 2021-07-06 07:00:00
종료: 2022-08-15 03:35:00


merged_table_valid.csv의 'Time' 데이터 (처음 20개):
2022-02-06 21:29:00
2022-02-07 10:43:00
2022-02-07 12:25:00
2022-02-07 13:13:00
2022-02-07 13:55:00
2022-02-07 14:22:00
2022-02-07 14:55:00
2022-02-07 15:51:00
2022-02-07 15:57:00
2022-02-07 16:50:00
2022-02-07 17:16:00
2022-02-07 17:25:00
2022-02-07 18:43:00
2022-02-07 18:52:00
2022-02-07 20:20:00
2022-02-07 21:51:00
2022-02-08 00:56:00
2022-02-08 02:02:00
2022-02-08 05:51:00
2022-02-08 15:54:00

총 고유 시간 개수: 1096

시간 범위:
시작: 

In [19]:
import pandas as pd
from datetime import timedelta
from tqdm import tqdm

# tqdm을 pandas 연산에 적용
tqdm.pandas()

# 데이터 로드
print("데이터 로딩 중...")
crrt_setting = pd.read_excel('CRRT_setting.xlsx')
merged_table = pd.read_csv('merged_table_valid.csv')

# 날짜/시간 형식 변환
print("날짜/시간 형식 변환 중...")
crrt_setting['[간호기록]기록작성일시'] = pd.to_datetime(crrt_setting['[간호기록]기록작성일시'])
merged_table['Time'] = pd.to_datetime(merged_table['Time'])

# 시간 매칭 함수
def find_closest_time(target_time, time_series, max_diff=timedelta(hours=3)):
    closest_time = min(time_series, key=lambda x: abs(x - target_time))
    if abs(closest_time - target_time) <= max_diff:
        return closest_time
    return pd.NaT

# merged_table의 각 시간에 대해 CRRT_setting에서 가장 가까운 시간 찾기
print("시간 매칭 중...")
merged_table['Matched_Time'] = merged_table['Time'].progress_apply(
    lambda x: find_closest_time(x, crrt_setting['[간호기록]기록작성일시'])
)

# 매칭된 결과 분석
matched_data = merged_table.dropna(subset=['Matched_Time'])
print(f"\n총 {len(matched_data)}개의 매칭된 데이터를 찾았습니다.")

# 시간 차이 계산
print("시간 차이 계산 중...")
matched_data['Time_Difference'] = (matched_data['Matched_Time'] - matched_data['Time']).abs()

# 결과 출력
print("\n매칭된 데이터 샘플 (처음 10개):")
print(matched_data[['Time', 'Matched_Time', 'Time_Difference', 'BFR', 'Pre', 'Replace', 'Dialysate']].head(10))

print("\n시간 차이 통계:")
print(matched_data['Time_Difference'].describe())

# 매칭된 데이터와 원본 CRRT_setting 데이터 병합
print("데이터 병합 중...")
merged_result = pd.merge(
    matched_data, 
    crrt_setting, 
    left_on='Matched_Time', 
    right_on='[간호기록]기록작성일시',
    how='left'
)

# 결과 저장
print("결과 저장 중...")
merged_result.to_csv('matched_crrt_data_with_tolerance.csv', index=False)
print("매칭된 데이터를 'matched_crrt_data_with_tolerance.csv' 파일로 저장했습니다.")

# 매칭되지 않은 데이터 분석
unmatched_data = merged_table[merged_table['Matched_Time'].isna()]
print(f"\n매칭되지 않은 데이터: {len(unmatched_data)}개")

if not unmatched_data.empty:
    print("\n매칭되지 않은 데이터 샘플 (처음 5개):")
    print(unmatched_data[['Time', 'BFR', 'Pre', 'Replace', 'Dialysate']].head())

# CRRT_setting에서 사용되지 않은 시간 분석
used_times = set(merged_result['[간호기록]기록작성일시'])
unused_times = crrt_setting[~crrt_setting['[간호기록]기록작성일시'].isin(used_times)]
print(f"\nCRRT_setting에서 매칭에 사용되지 않은 시간: {len(unused_times)}개")

if not unused_times.empty:
    print("\n사용되지 않은 시간 샘플 (처음 5개):")
    print(unused_times['[간호기록]기록작성일시'].head())

print("\n처리 완료!")

데이터 로딩 중...
날짜/시간 형식 변환 중...
시간 매칭 중...


  6%|▋         | 79/1224 [00:50<12:11,  1.57it/s]


KeyboardInterrupt: 

In [23]:
import pandas as pd
import numpy as np
from datetime import timedelta
from tqdm import tqdm

# tqdm을 pandas 연산에 적용
tqdm.pandas()

# 데이터 로드
print("데이터 로딩 중...")
crrt_setting = pd.read_excel('CRRT_setting.xlsx')
merged_table = pd.read_csv('merged_table_valid.csv')

# 날짜/시간 형식 변환
print("날짜/시간 형식 변환 중...")
crrt_setting['[간호기록]기록작성일시'] = pd.to_datetime(crrt_setting['[간호기록]기록작성일시'])
merged_table['Time'] = pd.to_datetime(merged_table['Time'])

# BFR, Pre, Replace, Dialysate 컬럼에 모두 값이 있는 행만 선택
columns_to_check = ['BFR', 'Pre', 'Replace', 'Dialysate']
merged_table_filtered = merged_table.dropna(subset=columns_to_check)

# ml/min, ml/h 단위 제거 및 숫자로 변환
for col in columns_to_check:
    merged_table_filtered[col] = merged_table_filtered[col].str.extract('(\d+)').astype(float)

print(f"필터링 후 남은 데이터: {len(merged_table_filtered)}행")

# 시간 매칭 함수
def find_closest_time(target_time, time_series, max_diff=timedelta(hours=3)):
    closest_time = min(time_series, key=lambda x: abs(x - target_time))
    if abs(closest_time - target_time) <= max_diff:
        return closest_time
    return pd.NaT

# merged_table의 각 시간에 대해 CRRT_setting에서 가장 가까운 시간 찾기
print("시간 매칭 중...")
merged_table_filtered['Matched_Time'] = merged_table_filtered['Time'].progress_apply(
    lambda x: find_closest_time(x, crrt_setting['[간호기록]기록작성일시'])
)

# 매칭된 결과 분석
matched_data = merged_table_filtered.dropna(subset=['Matched_Time'])
print(f"\n총 {len(matched_data)}개의 매칭된 데이터를 찾았습니다.")

# 시간 차이 계산
print("시간 차이 계산 중...")
matched_data['Time_Difference'] = (matched_data['Matched_Time'] - matched_data['Time']).abs()

# 결과 출력
print("\n매칭된 데이터 샘플 (처음 5개):")
print(matched_data[['Time', 'Matched_Time', 'Time_Difference', 'BFR', 'Pre', 'Replace', 'Dialysate']].head())

print("\n시간 차이 통계:")
print(matched_data['Time_Difference'].describe())

# 매칭된 데이터와 원본 CRRT_setting 데이터 병합
print("데이터 병합 중...")
merged_result = pd.merge(
    matched_data, 
    crrt_setting, 
    left_on='Matched_Time', 
    right_on='[간호기록]기록작성일시',
    how='left'
)

# 결과 출력
print("\n최종 매칭 결과 샘플 (처음 5개):")
print(merged_result[['Time', 'Matched_Time', 'Time_Difference', 'BFR', 'Pre', 'Replace', 'Dialysate', 'Attribute', 'Value']].head())

# 매칭되지 않은 데이터 분석
unmatched_data = merged_table_filtered[merged_table_filtered['Matched_Time'].isna()]
print(f"\n매칭되지 않은 데이터: {len(unmatched_data)}개")

if not unmatched_data.empty:
    print("\n매칭되지 않은 데이터 샘플 (처음 5개):")
    print(unmatched_data[['Time', 'BFR', 'Pre', 'Replace', 'Dialysate']].head())

# 결과 저장 (UTF-8 인코딩 적용)
print("\n결과를 CSV 파일로 저장 중...")
merged_result.to_csv('matched_crrt_data_filtered.csv', index=False, encoding='utf-8-sig')
print("매칭된 데이터를 'matched_crrt_data_filtered.csv' 파일로 저장했습니다. (UTF-8 인코딩)")

print("\n처리 완료!")

<>:25: SyntaxWarning: invalid escape sequence '\d'
<>:25: SyntaxWarning: invalid escape sequence '\d'
C:\Users\SNUH\AppData\Local\Temp\ipykernel_18936\3672733975.py:25: SyntaxWarning: invalid escape sequence '\d'
  merged_table_filtered[col] = merged_table_filtered[col].str.extract('(\d+)').astype(float)


데이터 로딩 중...


C:\Users\SNUH\AppData\Local\Temp\ipykernel_18936\3672733975.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_table_filtered[col] = merged_table_filtered[col].str.extract('(\d+)').astype(float)
C:\Users\SNUH\AppData\Local\Temp\ipykernel_18936\3672733975.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_table_filtered[col] = merged_table_filtered[col].str.extract('(\d+)').astype(float)
C:\Users\SNUH\AppData\Local\Temp\ipykernel_18936\3672733975.py:25: SettingWithCopyWarning: 
A value is t

날짜/시간 형식 변환 중...
필터링 후 남은 데이터: 114행
시간 매칭 중...


100%|██████████| 114/114 [00:57<00:00,  2.00it/s]


총 113개의 매칭된 데이터를 찾았습니다.
시간 차이 계산 중...

매칭된 데이터 샘플 (처음 5개):
                  Time        Matched_Time Time_Difference    BFR     Pre  \
0  2022-02-06 21:29:00 2022-02-06 21:30:00 0 days 00:01:00  120.0  1000.0   
24 2022-02-08 16:42:00 2022-02-08 16:30:00 0 days 00:12:00  120.0   650.0   
38 2022-02-09 01:11:00 2022-02-09 01:00:00 0 days 00:11:00  120.0   650.0   
49 2022-02-09 13:13:00 2022-02-09 13:01:00 0 days 00:12:00  120.0   650.0   
58 2022-02-10 10:01:00 2022-02-10 10:00:00 0 days 00:01:00  120.0   650.0   

    Replace  Dialysate  
0     200.0     1200.0  
24    200.0      850.0  
38    200.0      850.0  
49    200.0      850.0  
58    200.0      850.0  

시간 차이 통계:
count                          113
mean     0 days 00:07:16.991150442
std      0 days 00:07:46.860496439
min                0 days 00:00:00
25%                0 days 00:02:00
50%                0 days 00:05:00
75%                0 days 00:11:00
max                0 days 00:50:00
Name: Time_Difference, dtype: object


C:\Users\SNUH\AppData\Local\Temp\ipykernel_18936\3672733975.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_table_filtered['Matched_Time'] = merged_table_filtered['Time'].progress_apply(
C:\Users\SNUH\AppData\Local\Temp\ipykernel_18936\3672733975.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_data['Time_Difference'] = (matched_data['Matched_Time'] - matched_data['Time']).abs()
